<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week1_PH_Python_Colab_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 1 Practical (Python, Colab)
**Topic:** Introduction; survey of paradigms; **grammar & AST basics**  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This lab sets up your environment and gets you comfortable with defining a **grammar** and building an **AST**.
> The code cells are **commented line‑by‑line** to reinforce understanding.

### Learning goals (≈60 minutes)
- Understand what an **AST** is and why we separate syntax from meaning.  
- Build a minimal AST for a tiny expression language and pretty‑print it.  
- Practice **manual AST construction** for a few expressions and run quick checks.

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"  # <-- type your full name
STUDENT_ID   = "Matric/ID here"            # <-- type your Matric/ID
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)  # echo back for logs

In [ ]:
# ✅ Environment check (we use Python 3.10+ for pattern matching)
import sys                                   # import the system module to access version info
major, minor = sys.version_info[:2]          # extract major and minor version numbers
assert (major, minor) >= (3, 10), (          # assert that Python version is at least 3.10
    f"Need Python 3.10+, found {major}.{minor}")
print(f"Python {major}.{minor} OK — match/case available.")  # success message

In [ ]:
# =====================================
# Part 1 — Minimal AST (line-by-line comments)
# =====================================
from __future__ import annotations       # allow forward references in type hints
from dataclasses import dataclass        # dataclass reduces boilerplate for small records
from typing import Union, Dict           # basic typing utilities

# --- Leaf nodes ---
@dataclass(frozen=True)                  # frozen=True makes node immutable (like real ASTs)
class Int:                               # integer literal node
    n: int                               # field: the integer value

@dataclass(frozen=True)
class Bool:                               # boolean literal node
    b: bool                              # field: the boolean value

@dataclass(frozen=True)
class Var:                                # variable node
    x: str                                # field: variable name

# --- Expression nodes (binary ops, conditionals, let-binding) ---
@dataclass(frozen=True)
class Add:                                # addition: a + b
    a: "Expr"                             # left operand (an Expr)
    b: "Expr"                             # right operand (an Expr)

@dataclass(frozen=True)
class Sub:                                # subtraction: a - b
    a: "Expr"
    b: "Expr"

@dataclass(frozen=True)
class Mul:                                # multiplication: a * b
    a: "Expr"
    b: "Expr"

@dataclass(frozen=True)
class Eq:                                 # equality: a == b
    a: "Expr"
    b: "Expr"

@dataclass(frozen=True)
class If:                                 # conditional: if c then t else e
    c: "Expr"                             # condition
    t: "Expr"                             # then branch
    e: "Expr"                             # else branch

@dataclass(frozen=True)
class Let:                                # let-binding: let x = e1 in e2
    x: str                                # name being bound
    e1: "Expr"                            # bound expression
    e2: "Expr"                            # body where x is in scope

# --- Type alias for any expression node ---
Expr = Union[Int, Bool, Var, Add, Sub, Mul, Eq, If, Let]  # union of all node types

In [ ]:
# =====================================
# Part 2 — Pretty-printer for the AST (line-by-line comments)
# =====================================
def pretty(e: Expr) -> str:               # function returning a string representation
    match e:                               # Python 3.10 structural pattern matching on node shape
        case Int(n):                       # if the node is Int(...)
            return str(n)                  # print the integer value
        case Bool(b):                      # Bool(...)
            return str(b).lower()          # use lowercase 'true'/'false' for readability
        case Var(x):                       # Var(...)
            return x                       # print the variable name
        case Add(a, b):                    # Add(left, right)
            return f"({pretty(a)} + {pretty(b)})"   # recursively pretty-print subexpressions
        case Sub(a, b):                    # Sub(left, right)
            return f"({pretty(a)} - {pretty(b)})"
        case Mul(a, b):                    # Mul(left, right)
            return f"({pretty(a)} * {pretty(b)})"
        case Eq(a, b):                     # Eq(left, right)
            return f"({pretty(a)} == {pretty(b)})"
        case If(c, t, e):                  # If(condition, then, else)
            return f"if {pretty(c)} then {pretty(t)} else {pretty(e)}"
        case Let(x, e1, e2):               # Let(name, bound, body)
            return f"let {x} = {pretty(e1)} in {pretty(e2)}"

In [ ]:
# =====================================
# Part 3 — Manual AST construction drills (line-by-line comments)
# =====================================
# (1) Build AST for:      (1 + 2) * 3
ast1 = Mul(                      # outermost node is multiplication
    Add(Int(1), Int(2)),         # left side is 1 + 2
    Int(3)                       # right side is 3
)
print("Ex1:", pretty(ast1))      # show the pretty-printed form

# (2) Build AST for:      let x = 2 in x * (3 + 4)
ast2 = Let(                      # Let-binding node
    "x",                         # bind the name x
    Int(2),                      # x = 2
    Mul(Var("x"), Add(Int(3), Int(4)))  # body: x * (3 + 4)
)
print("Ex2:", pretty(ast2))

# (3) Build AST for:      if true then 1 else 0
ast3 = If(                       # If node
    Bool(True),                  # condition: true
    Int(1),                      # then branch
    Int(0)                       # else branch
)
print("Ex3:", pretty(ast3))

In [ ]:
# =====================================
# Part 4 — Tiny evaluator (big-step) with **line-by-line comments**
# =====================================
Value = Union[int, bool]          # values are ints or bools in this tiny language
Env   = Dict[str, Value]          # environment mapping names to values

class RuntimeErrorPL(Exception):   # custom exception for runtime errors
    ...

def eval_ast(e: Expr, env: Env | None = None) -> Value:  # evaluator function
    env = {} if env is None else dict(env)               # use empty env by default (copy to avoid mutation)
    match e:                                             # dispatch by node shape
        case Int(n):                                     # literal integer
            return n                                     # evaluate to itself
        case Bool(b):                                    # literal boolean
            return b                                     # evaluate to itself
        case Var(x):                                     # variable reference
            if x in env:                                 # look up the variable name
                return env[x]                            # return its value
            raise RuntimeErrorPL(f"unbound variable {x}")  # error if not found
        case Add(a, b):                                  # addition
            return eval_ast(a, env) + eval_ast(b, env)   # evaluate both sides and add
        case Sub(a, b):                                  # subtraction
            return eval_ast(a, env) - eval_ast(b, env)   # evaluate and subtract
        case Mul(a, b):                                  # multiplication
            return eval_ast(a, env) * eval_ast(b, env)   # evaluate and multiply
        case Eq(a, b):                                   # equality
            return eval_ast(a, env) == eval_ast(b, env)  # evaluate and compare
        case If(c, t, e2):                               # conditional
            return eval_ast(t, env) if eval_ast(c, env) else eval_ast(e2, env)  # choose branch
        case Let(x, e1, e2):                             # let-binding
            v1 = eval_ast(e1, env)                       # evaluate the bound expression
            env2 = dict(env)                             # create a new extended environment
            env2[x] = v1                                 # bind x to its value
            return eval_ast(e2, env2)                    # evaluate the body with extended env

In [ ]:
# =====================================
# Part 5 — Quick checks (line-by-line comments)
# =====================================
print("Run Ex1:", eval_ast(ast1, {}))                    # should print 9
print("Run Ex2:", eval_ast(ast2, {}))                    # should print 14
print("Run Ex3:", eval_ast(ast3, {}))                    # should print 1
assert eval_ast(ast1, {}) == 9                           # sanity check
assert eval_ast(ast2, {}) == 14                          # sanity check
assert eval_ast(ast3, {}) == 1                           # sanity check
print("ok  - all Week 1 checks passed")                  # final message

### 🧪 Your Turn (10–15 minutes)
1) Build the AST for `(10 - 3) * (2 + 5)` and evaluate it.  
2) Build the AST for `let y = 7 in if (y == 7) then (y * 2) else 0` and evaluate it.  
3) **(Stretch)** Add a new node `Neg(e)` to represent unary minus and extend the pretty‑printer and evaluator.

### ✍️ Reflection (2–3 sentences)
- Why is it useful to work with an **AST** rather than raw strings when building interpreters or compilers?  
- Where do you see the **separation of concerns** between parsing, AST, and evaluation?

In [ ]:
# 📝 Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")  # current timestamp for record
submission = {
  "student_name": STUDENT_NAME,
  "student_id": STUDENT_ID,
  "timestamp": stamp,
  "checks": ["ast1", "ast2", "ast3"],
  "reflection": "(fill in here)"
}
with open("week1_submission.json", "w") as f:  # write JSON file
  json.dump(submission, f, indent=2)
print("Saved week1_submission.json — upload with your notebook.")